In [ ]:
# Start writing code here...

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import multinomial, dirichlet

In [2]:
df = pd.read_excel('Primera_vuelta_2021.xlsx')
datos2021 = df.to_numpy()[:, 3:7]

In [29]:
datos2021

array([[  96,   80,   32,   48],
       [ 115,   51,   85,  178],
       [ 548,  737,  357,  580],
       [ 209,  316,  178,  352],
       [ 117,  137,   48,  234],
       [  95,   65,   57,  121],
       [ 143,  215,  143,  215],
       [ 166,  300,  236,  298],
       [ 192,  480,  384,  288],
       [  56,  175,  210,   84],
       [1214, 2971, 3305, 1054],
       [ 269,  707,  827,  381],
       [ 302,  668,  523,  242],
       [ 357,  463,  612,  379],
       [ 251,  587,  737,  357],
       [ 241,  425,  592,  249],
       [ 157,  307,  427,  166],
       [ 222,  294,  438,  169],
       [ 500,  442,  768,  343],
       [1587, 1183, 2142,  948],
       [ 618,  271,  790,  466],
       [ 576,  169,  575,  260],
       [2265,  730, 2066,  951],
       [7741, 2540, 6373, 2763]])

In [3]:
from scipy.stats import dirichlet

y = datos2021[0]
p = np.full(4, 1/4)
alpha = 1000*p+y
dirichlet.rvs(alpha)
    

array([[0.27922469, 0.27150239, 0.20262768, 0.24664524]])

In [4]:
def modelo2021(data, N):
    post = lambda p, alpha: dirichlet.pdf(p, alpha)
    weeks = np.size(data, axis=0)
    muestra = np.zeros((N, weeks+1, 4))
    prediccion = np.zeros((N, 4))
    prior = np.full(4, 0.25)
    d = 0.5
    for i in range(N):
        muestra[i, 0] = prior
        for week in range(weeks):
            p = muestra[i, week]
            y = data[week]
            p_prueba = np.random.uniform(np.max(p-d,0), p+d, size=4)
            p_prueba = p_prueba/p_prueba.sum()
            alpha = 1000 * p + y
            if post(p_prueba, alpha) / post(p, alpha) > np.random.uniform():
                muestra[i, week+1] = p_prueba
            else:
                muestra[i, week+1] = p
        prediccion[i] = muestra[i, -1]
    return muestra, prediccion
        
    

In [ ]:
def modelo2021(data, N):
    weeks = np.size(data, axis=0)
    muestra = np.zeros((N, weeks+1, 4))
    prediccion = np.zeros((N, 4))
    prior = np.full(4, 0.25)
    d = 0.5
    for i in range(N):
        prob = prior
        for week in range(weeks):
            y = data[week]
            alpha = 1000 * prob + y
            prob = dirichlet.rvs(alpha)
            muestra[i, week] = multinomial.rvs(prob)
        prediccion[i] = muestra[i, -1]
    return muestra, prediccion

In [6]:
p = np.array([0.4,0.1,0.35,0.15])
p = p/p.sum()
a = np.array([0.25,0.25,0.25,0.25])
y = np.array([96,80,32,48])
x = np.power(p,a*0.001+y-1)
x.prod()


2.1748448547226404e-170

In [5]:
np.random.seed(756)
muestra, prediccion = modelo2021(datos2021, 10_000)

ValueError: Each entry in 'x' must be greater than or equal to zero.

In [23]:
muestra

array([[[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        ...,
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        ...,
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        ...,
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       ...,

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        ...,
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25]],

       [[0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        [0.25, 0.25, 0.25, 0.25],
        .

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=d24ecb2b-b17d-41bb-82ac-38080033d82c' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>